#### CASO NÓMINAS

Teniendo un dataset con las siguientes columnas:
- COD_PERSONA	Identificador cliente
- COD_EDAD	Edad del cliente
- XTI_IDEFISCO	Tipo de identificador fiscal
- FEC_CANCEL	Fecha cancelación cliente
- COD_POSTAL	Código postal
- IMP_NOMINA	Importe Nómina Mes Análisis
- IMP_PENSION	Importe Pensión Mes Análisis
- IMP_DESEMPL	Importe Desempleo Mes Análisis

Queremos resolver dos preguntas relacionadas con la tenencia de ingresos:
* 1ª Pregunta: ¿qué importes medios de nómina, pensión o desempleo tengo en mi BBDD de Nov 21?
* 2ª Pregunta: queremos saber cómo ha evolucionado la base de nóminas de Nov20 a Nov21 y cuál de las dos campañas que tenemos vigentes ha funcionado mejor: nuevas nóminas o captación de clientes

### Abrir los fichero de datos

In [ ]:
import pandas as pd

In [ ]:
df_2021 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Transformación Digital/4. CASO NÓMINAS.xlsx - BASE CLIENTES NOV21.csv')
df_2021

,COD_PERSONA,COD_EDAD,XTI_IDEFISCO,FEC_CANCEL,COD_POSTAL,IMP_NOMINA,IMP_PENSION,IMP_DESEMPL
0,55574,48,1,00/00/0000,41010,1700,0,0
1,62886,44,1,00/00/0000,41010,1756,0,0
2,60718,26,1,00/00/0000,39300,0,0,93
3,41027,29,1,00/00/0000,41010,444,0,0
4,15613,33,1,00/00/0000,41010,0,0,318
...,...,...,...,...,...,...,...,...
994,52516,29,1,00/00/0000,28100,0,0,0
995,66964,63,1,00/00/0000,39300,1969,0,0
996,45022,26,1,00/00/0000,28100,561,0,0
997,59516,34,1,00/00/0000,41010,2933,0,0


In [ ]:
df_2020 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Transformación Digital/4. CASO NÓMINAS.xlsx - BASE CLIENTES NOV20.csv')
df_2020

,COD_PERSONA,COD_EDAD,XTI_IDEFISCO,FEC_CANCEL,COD_POSTAL,IMP_NOMINA,IMP_PENSION,IMP_DESEMPL
0,92047,79,1,00/00/0000,39300,0,600,0
1,12415,42,1,00/00/0000,39300,0,1200,0
2,31103,21,1,00/00/0000,23678,0,0,0
3,91094,31,1,00/00/0000,28034,0,1100,0
4,81707,48,1,00/00/0000,28034,0,800,0
...,...,...,...,...,...,...,...,...
944,46018,22,1,00/00/0000,28100,1922,0,0
945,29889,74,1,00/00/0000,28100,0,0,2116
946,11778,67,1,00/00/0000,28100,520,0,0
947,77336,49,1,00/00/0000,41010,0,250,0


### Filtros
Es un problema de clientes particulares, tendríamos que filtrar a personas jurídicas pero parece no haber
Sí hay que filtrar clientes cancelados para sacar los datos que me piden

In [ ]:
import numpy as np

In [ ]:
def preparacion_dataset(df):
  df = df[(df['XTI_IDEFISCO'] != 2) & (df['XTI_IDEFISCO'] != 3)]
  df = df[(df['FEC_CANCEL'] == '00/00/0000')]
  df['NOMINA'] = np.where(df['IMP_NOMINA'] > 0, 1, 0)
  df['PENSION'] = np.where(df['IMP_PENSION'] > 0, 1, 0)
  df['DESEMPL'] = np.where(df['IMP_DESEMPL'] > 0, 1, 0)
  return df

In [ ]:
df_2021 = preparacion_dataset(df_2021)
df_2021 = df_2021.drop_duplicates(subset='COD_PERSONA', keep="last")

In [ ]:
print('DATOS DE NÓMINAS DE 2021')
df_2021.agg({'NOMINA': 'sum',
                               'IMP_NOMINA': ['sum', lambda x: x.sum() / float(len(df_2021[(df_2021['NOMINA'] == 1)]))],
                               'PENSION': 'sum',
                               'IMP_PENSION': ['sum', lambda x: x.sum() / float(len(df_2021[(df_2021['PENSION'] == 1)]))],
                               'DESEMPL': 'sum',
                               'IMP_DESEMPL': ['sum', lambda x: x.sum() / float(len(df_2021[(df_2021['DESEMPL'] == 1)]))],})

DATOS DE NÓMINAS DE 2021


,NOMINA,IMP_NOMINA,PENSION,IMP_PENSION,DESEMPL,IMP_DESEMPL
sum,538.0,794497.000000,218.0,197500.000000,135.0,165492.000000
<lambda>,NaN,1476.760223,NaN,905.963303,NaN,1225.866667


In [ ]:
df_2020 = preparacion_dataset(df_2020)
df_2020 = df_2020.drop_duplicates(subset='COD_PERSONA', keep="last")

In [ ]:
print('DATOS DE NÓMINAS DE 2020')
df_2020.groupby('NOMINA').agg({'NOMINA': 'sum',
                               'IMP_NOMINA': ['sum', lambda x: x.sum() / float(len(df_2021[(df_2021['NOMINA'] == 1)]))],
                               'PENSION': 'sum',
                               'IMP_PENSION': ['sum', lambda x: x.sum() / float(len(df_2021[(df_2021['PENSION'] == 1)]))],
                               'DESEMPL': 'sum',
                               'IMP_DESEMPL': ['sum', lambda x: x.sum() / float(len(df_2021[(df_2021['DESEMPL'] == 1)]))],})

DATOS DE NÓMINAS DE 2020


NOMINA IMP_NOMINA              PENSION IMP_PENSION             DESEMPL  \
          sum        sum   <lambda_0>     sum         sum  <lambda_0>     sum   
NOMINA                                                                          
0           0          0     0.000000     233      217525  997.821101     146   
1         452     664599  1235.314126       0           0    0.000000       0   

       IMP_DESEMPL               
               sum   <lambda_0>  
NOMINA                           
0           182400  1351.111111  
1                0     0.000000

#### Cruzamos ambas bases de datos
Queremos saber no sólo si el cliente estaba en 2020 sino también si tenía en esas fechas Nómina, Pensión o Desempleo.

In [ ]:
df_all = df_2020.merge(df_2021, on=['COD_PERSONA'],
                   how='outer', indicator=True, suffixes=('_2020', '_2021'))
print(df_all.groupby('_merge').agg({'_merge': 'count'}))
print(len(df_all))

            _merge
_merge            
left_only       22
right_only      71
both           895
988


#### Clientes en nómina en 2020

In [ ]:
print("Clientes en nómina en 2020: ", df_all[(df_all['NOMINA_x'] == 1)].shape[0])

Clientes en nómina en 2020:  452


#### Clientes dados de baja de 2020 a 2021

In [ ]:
print("Clientes dados de baja de 2020 a 2021: ", df_all[(df_all['NOMINA_x'] == 1) & (df_all['_merge'] == 'left_only')].shape[0])

Clientes dados de baja de 2020 a 2021:  7


#### Clientes que ya no tienen nómina en 2021

In [ ]:
print("Clientes que ya no tienen nómina en 2021: ", df_all[(df_all['NOMINA_x'] == 1) & (df_all['_merge'] == 'both') & ((df_all['NOMINA_y'] == 0))].shape[0])

Clientes que ya no tienen nómina en 2021:  1


#### Clientes que permanecen en nómina en 2021

In [ ]:
print("Clientes que permanecen en nómina en 2021: ", df_all[(df_all['NOMINA_x'] == 1) & (df_all['_merge'] == 'both') & (df_all['NOMINA_y'] == 1)].shape[0])

Clientes que permanecen en nómina en 2021:  444


#### Clientes dados de alta en 2021 y con nómina

In [ ]:
print("Clientes con nómina dados de alta en 2021: ", df_all[(df_all['_merge'] == 'right_only') & (df_all['NOMINA_y'] == 1)].shape[0])

Clientes con nómina dados de alta en 2021:  35


#### Clientes que pasan a tener nómina en 2021 y que en 2020 tenían Pensión

In [ ]:
print("Clientes que pasan a tener nómina en 2021 (PENSIÓN): ", df_all[(df_all['_merge'] == 'both') & (df_all['NOMINA_y'] == 1) & (df_all['PENSION_x'] == 1) & (df_all['NOMINA_x'] == 0)].shape[0])

Clientes que pasan a tener nómina en 2021 (PENSIÓN):  28


#### Clientes que pasan a tener nómina en 2021 y que en 2020 tenían Desempleo

In [ ]:
print("Clientes que pasan a tener nómina en 2021 (DESEMPLEO): ", df_all[(df_all['_merge'] == 'both') & (df_all['NOMINA_y'] == 1) & (df_all['DESEMPL_x'] == 1) & (df_all['NOMINA_x'] == 0)].shape[0])

Clientes que pasan a tener nómina en 2021 (DESEMPLEO):  13


#### Clientes que pasan a tener nómina en 2021 (únicamente nuevas nóminas, descartando aquellos que tenían desempleo o pensión en 2020

In [ ]:
print("Clientes que pasan a tener nómina en 2021 (NUEVAS NÓMINAS): ", df_all[(df_all['_merge'] == 'both') & (df_all['NOMINA_y'] == 1) & (df_all['DESEMPL_x'] == 0) & (df_all['PENSION_x'] == 0) & (df_all['NOMINA_x'] == 0)].shape[0])

Clientes que pasan a tener nómina en 2021 (NUEVAS NÓMINAS):  18


#### Clientes totales con nómina en 2021

In [ ]:
print("Clientes con nómina en 2021: ", df_all[(df_all['_merge'] != 'left_only') & (df_all['NOMINA_y'] == 1)].shape[0])

Clientes con nómina en 2021:  538


In [ ]:
df_all.groupby('NOMINA_y').agg({'NOMINA_x': 'sum',
            'NOMINA_y': 'sum',
            'PENSION_x': 'sum',
            'PENSION_y': 'sum',
            'DESEMPL_x': 'sum',
            'DESEMPL_y': 'sum'})

,NOMINA_x,NOMINA_y,PENSION_x,PENSION_y,DESEMPL_x,DESEMPL_y
NOMINA_y,,,,,,
0.0,1.0,0.0,197.0,218.0,128.0,135.0
1.0,444.0,538.0,28.0,0.0,13.0,0.0
